In [54]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time

# Test Implementations (Naive):

In [55]:
###### Specify parameters for problem ###### 
T = 5 # Time Horizon
toll_arr = 20*np.linspace(0, 1, 21)[0:] # Toll discretization
print('Toll:\n {}'.format(toll_arr))
print()

B_arr = np.linspace(toll_arr[0], toll_arr[-3]*T,19) # Budget discretization 
print('B:\n {}'.format(B_arr))
print()

num_total = 8000 # number of users
num_eligible = int( num_total*.17) # number of eligible users
num_ineligible = num_total-num_eligible # number of ineligible users

###### VoT information ###### 
vot_all = np.sort(np.round(np.random.choice(vots, size = num_total, p=vot_p),2)/60) # generate baseline VoT for all users
vot_all_days = np.empty(shape=(num_total,T)) 
count=0
for v in vot_all:
    if count < num_eligible:
        vot_all_days[count,:]=[v for t in range(T)] # VoT of eligible users does not vary over time periods
    else:
        vot_all_days[count,:]=list((1+0.25*(np.random.random_sample(size=T)-0.5))*v) # VoT of ineligible users varies over time periods
    count+=1
    
vot_eligible = vot_all_days[:num_eligible,:]
vot_ineligible= vot_all_days[num_eligible:,:]

print('Eligible vot range:\n {} ({} $/hr), {} ({} $/hr)'.format(np.min(vot_eligible),60*np.min(np.round(vot_eligible,2)),np.max(np.round(vot_eligible,2)),60*np.max(np.round(vot_eligible,2))))
print()
print('Ineligible vot range:\n {}({} $/hr), {} ({} $/hr)'.format(np.min(np.round(vot_ineligible,2)),60*np.min(np.round(vot_ineligible,2)),np.max(np.round(vot_ineligible,2)),60*np.max(np.round(vot_ineligible,2))))
print()
print('Mean vot:\n {} $/hr; median: {} $/hr'.format(60*np.round(np.mean(vot_all_days),2),60*np.round(np.median(vot_all_days),2)))

Toll:
 [ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20.]

B:
 [ 0.  5. 10. 15. 20. 25. 30. 35. 40. 45. 50. 55. 60. 65. 70. 75. 80. 85.
 90.]



NameError: name 'vots' is not defined

In [56]:
def OptPL_budget(toll, B, T = T, num_eligible = num_eligible, num_ineligible = num_ineligible, 
            vot_ineligible = vot_ineligible, alpha = bpr_true_alpha, fftt_1 = bpr_true_fftt, 
              fftt_2 = bpr_true_fftt, c_1 = bpr_true_capacity, c_2 = bpr_true_capacity, 
             a = const_multiplier, cap_thresh = cap_thresh_multiplier, b_1 = apx_slope, b_2 = apx_slope):
    """
    Function to solve convex optimization problem given a particular toll and budget value
    """
    
    # Initialize model
    m1 = gp.Model()
    # Add variables to model
    users_in = m1.addVars(num_ineligible, 3, T, name = 'ineligible_val')
    edge_flows = m1.addVars(2, T, name = 'edge_flows')
    users_el = m1.addVars(3, T, name = 'eligible_allocation')
    
    # Add variables for piecewise affine approximation
    eps_flows = m1.addVars(2, T, name = 'eps_flows')

    ## Add constraints to Model 
    
    # Total edge flow, express lane:
    m1.addConstrs((edge_flows[0, t] == sum(users_in[i, 0, t] for i in range(num_ineligible)) + \
                  users_el[0, t] + sum(users_in[i, 1, t] for i in range(num_ineligible)) + \
                  users_el[1, t] for t in range(T)))
    # Total edge flow, general purpose lane:
    m1.addConstrs((edge_flows[1, t] == sum(users_in[i, 2, t] for i in range(num_ineligible)) + \
                  users_el[2, t] for t in range(T)))
    
    # Non-negativity of flows, and zero flow for "eligible users in ineligible groups":
    m1.addConstrs((users_in[i, j, t]>=0 for i in range(num_ineligible) for j in range(3) for t in range(T)))
    m1.addConstrs((users_in[i, 0, t]==0 for i in range(num_ineligible) for t in range(T)))
    m1.addConstrs((users_el[j, t]>=0 for j in range(3) for t in range(T)))
    
    # Every eligible and ineligible user is assigned to one of three options: \
    # (express paying, express with credit, non-express)
    m1.addConstrs((sum(users_in[i, j, t] for j in range(3)) == 1 for i in range(num_ineligible) for t in range(T)))
    m1.addConstrs((sum(users_el[j, t] for j in range(3)) == 1 for t in range(T)))
     
    # Budget constraint satisfaction:
    if toll != 0:
        m1.addConstr((sum(toll*users_el[0, t] for t in range(T)) <= B*num_eligible))
        
    # Piecewise affine approximation:
    m1.addConstrs((eps_flows[j, t] >= 0 for j in range(2) for t in range(T))) # Must be at least 0
    m1.addConstrs((eps_flows[0, t] >= edge_flows[0, t] - cap_thresh*c_1 for t in range(T))) # Must be at least flow - capacity
    m1.addConstrs((eps_flows[1, t] >= edge_flows[1, t] - cap_thresh*c_2 for t in range(T))) # Must be at least flow - capacity

    ### Frank: Unmodified below:
    
    # Set Objective
    m1.setObjective(sum( a*fftt_1*(edge_flows[0, t]) + b_1*(eps_flows[0, t]**2)/2 \ 
                        + a*fftt_2*(edge_flows[1, t]) + b_2*(eps_flows[1, t]**2)/2 \
                        + sum( toll*users_in[i,1,t]/vot_ineligible[i,t] for i in range(num_ineligible)) \
        for t in range(T)), GRB.MINIMIZE)
    
    m1.update()
    
    return m1

SyntaxError: unexpected character after line continuation character (4170757482.py, line 51)

In [11]:
### Frank: Unmodified below:

def _extract_solution(m, num_ineligible = num_ineligible):
    """
    Get solution from optimization model
    """
    users_in = [v.x for v in m.getVars() if v.VarName.find("ineligible_val") != -1]
    users_el = [v.x for v in m.getVars() if v.VarName.find("eligible_allocation") != -1]
    edge_flows = [v.x for v in m.getVars() if v.VarName.find("edge_flows") != -1]
    eps_flows = [v.x for v in m.getVars() if v.VarName.find("eps_flows") != -1]
    
    users_in = np.reshape(users_in, (num_ineligible, 3, T))
    users_el = np.reshape(users_el, (3, T))
    edge_flows = np.reshape(edge_flows, (2, T))
    eps_flows = np.reshape(eps_flows, (2, T))

    solution = {
        "users_in": users_in,
        "users_el": users_el,
        "edge_flows": edge_flows,
        "eps_flows": eps_flows
    }
    return solution

# Sample Optimization Problem:

## Gurobi:

In [12]:
# # Trying to minimize f(x) = (x-2)^2 + 1 = x^2 - 4x + 5

# m_test = gp.Model()
# x = m_test.addVars(1, name = 'x')
# m_test.setObjective( x[0]**2 - 4*x[0] + 5, GRB.MINIMIZE)
# # m_test.update()
# m_test.optimize()


In [13]:
# m_test.getVars()[0].x

In [50]:
time_1 = time.time()

toll = 1.0
budget = 3.0
demand_multiplier = [25,75]

# num_ineligible_trunc = num_ineligible

# Truncating num_ineligible to enable comparison with CVXPY (which is very, very slow):
num_ineligible_trunc = 500

m_budget = OptPL_budget(toll, budget, T = T, num_eligible = num_eligible, num_ineligible = num_ineligible_trunc, 
            vot_ineligible = vot_ineligible, alpha = bpr_true_alpha, fftt_1 = bpr_true_fftt, 
              fftt_2 = bpr_true_fftt, c_1 = bpr_true_capacity*demand_multiplier[0], c_2 = bpr_true_capacity*demand_multiplier[1], 
             a = const_multiplier, cap_thresh = cap_thresh_multiplier, b_1 = apx_slope/demand_multiplier[0], b_2 = apx_slope/(demand_multiplier[1]))

m_budget.optimize()

time_2 = time.time()
print("Run time:", time_2 - time_1)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 22531 rows, 7535 columns and 40060 nonzeros
Model fingerprint: 0xab24e3be
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+00, 2e+01]
  QObjective range [4e-03, 1e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+03]
Presolve removed 22531 rows and 7535 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.02 seconds (0.01 work units)
Optimal objective 4.93522388e+04
Run time: 0.5195481777191162


In [51]:
# m_budget.getVars()
# solution_set = _extract_solution(m_budget, num_ineligible_trunc)

m_budget.getVars()

# list_from_gurobi = [v.x for v in m_budget.getVars()]
# list_from_gurobi

users_in = [v.x for v in m_budget.getVars() if v.VarName.find("ineligible_val") != -1]
users_el = [v.x for v in m_budget.getVars() if v.VarName.find("eligible_allocation") != -1]
edge_flows = [v.x for v in m_budget.getVars() if v.VarName.find("edge_flows") != -1]
eps_flows = [v.x for v in m_budget.getVars() if v.VarName.find("eps_flows") != -1]
    
users_in = np.reshape(users_in, (num_ineligible_trunc, 3, T))
users_el = np.reshape(users_el, (3, T))
edge_flows = np.reshape(edge_flows, (2, T))
eps_flows = np.reshape(eps_flows, (2, T))

# solution = {
#     "users_in": users_in,
#     "users_el": users_el,
#     "edge_flows": edge_flows,
#     "eps_flows": eps_flows
# }

solution_gurobi = {
    "users_in": users_in,
    "users_el": users_el,
    "edge_flows": edge_flows,
    "eps_flows": eps_flows
}


## CVXPY:

## Case 1, Budget:

In [53]:
alpha = bpr_true_alpha
fftt_1 = bpr_true_fftt
fftt_2 = bpr_true_fftt
c_1 = bpr_true_capacity
c_2 = bpr_true_capacity
a = const_multiplier
cap_thresh = cap_thresh_multiplier
b_1 = apx_slope
b_2 = apx_slope

# num_ineligible_trunc = 10

time_1 = time.time()

users_in = []
for i in range(num_ineligible_trunc):
    users_in += [cp.Variable((3, T))]
edge_flows = cp.Variable((2, T))
users_el = cp.Variable((3, T))
eps_flows = cp.Variable((2, T))

constraints = []

# Total edge flow, express lane:
print("Adding total edge flow, express lane constraints.")
constraints += [(edge_flows[0, t] == sum(users_in[i][0, t] for i in range(num_ineligible_trunc)) + \
              users_el[0, t] + sum(users_in[i][1, t] for i in range(num_ineligible_trunc)) + \
              users_el[1, t]) for t in range(T)]

# Total edge flow, general purpose lane:
print("Adding total edge flow, general purpose lane constraints.")
constraints += [(edge_flows[1, t] == sum(users_in[i][2, t] for i in range(num_ineligible_trunc)) + \
              users_el[2, t]) for t in range(T)]

# Non-negativity of flows, and zero flow for "eligible users in ineligible groups:
print("Adding non-negativity of flows constraints.")
constraints += [(users_in[i][j, t] >= 0) for i in range(num_ineligible_trunc) for j in range(3) for t in range(T)]
constraints += [(users_in[i][0, t] == 0) for i in range(num_ineligible_trunc) for t in range(T)]
constraints += [(users_el[j, t] >= 0) for j in range(3) for t in range(T)]

# Every eligible and ineligible user is assigned to one of three options: \
# (express paying, express with credit, non-express)
print("Adding user assignment constraints.")
constraints += [(sum(users_in[i][j, t] for j in range(3)) == 1) for i in range(num_ineligible_trunc) for t in range(T)]
constraints += [(sum(users_el[j, t] for j in range(3)) == 1) for t in range(T)]

# Budget constraint satisfaction:
print("Adding budget satisfaction constraints.")
if toll != 0:
    constraints += [sum(toll*users_el[0, t] for t in range(T)) <= budget * num_eligible] 

# Piecewise affine approximation:
print("Adding piecewise affine approximation constraints.")
constraints += [(eps_flows[j, t] >= 0) for j in range(2) for t in range(T)]
constraints += [(eps_flows[0, t] >= edge_flows[0, t] - cap_thresh*c_1) for t in range(T)]
constraints += [(eps_flows[1, t] >= edge_flows[1, t] - cap_thresh*c_2) for t in range(T)]

# Set Objective
print("Setting objective.")
objective = cp.Minimize(sum( a*fftt_1*(edge_flows[0, t]) + b_1*(eps_flows[0, t]**2)/2 \
                    + a*fftt_2*(edge_flows[1, t]) + b_2*(eps_flows[1, t]**2)/2 \
                    + sum( toll*users_in[i][1,t]/vot_ineligible[i,t] for i in range(num_ineligible_trunc)) \
    for t in range(T)))


# Solve problem
print("Forming problem.")
prob = cp.Problem(objective, constraints)

print("Solving problem.")
result = prob.solve()

time_2 = time.time()
print()
print("Run time:", time_2 - time_1)

NameError: name 'bpr_true_alpha' is not defined

In [53]:

users_in_values_cvxpy = np.zeros((num_ineligible_trunc, 3, T))
for i in range(num_ineligible_trunc):
    users_in_values_cvxpy[i, :, :] = users_in[i].value

solution_cvxpy = {
    "users_in": users_in_values_cvxpy,
    "users_el": users_el.value,
    "edge_flows": edge_flows.value,
    "eps_flows": eps_flows.value
}


In [54]:
print(solution_gurobi["edge_flows"])
print()
print(solution_cvxpy["edge_flows"])
print("\n")

print(solution_gurobi["eps_flows"])
print()
print(solution_cvxpy["eps_flows"])

[[  1.   1.   1.   1.   1.]
 [500. 500. 500. 500. 500.]]

[[244.0233496  244.12728384 244.23866486 244.19237969 244.19254432]
 [256.97664829 256.87270872 256.7613343  256.80762163 256.80745451]]


[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

[[192.31250948 192.41678911 192.52772812 192.48128388 192.48160163]
 [205.26548894 205.16120415 205.05027097 205.09671735 205.09639722]]


## Case 2, Discount:

## Sample CVXPY code:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
# for i in range(2):
#     constraints += [x[i] >= 2]
# constraints += [x[i] >=2 for i in range(2)]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()



In [ ]:
# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print()
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print("constraints[0].dual_value:", constraints[0].dual_value)

# LP, Comparison between CVXPY and Gurobi:

In [135]:
c_1 = np.array([-1, 1])
c_2 = np.array([-1, -1])

In [153]:
x_cvxpy = cp.Variable(2)
prob = cp.Problem(cp.Minimize(x_cvxpy[0]), [c_1 @ x_cvxpy <= -3, c_2 @ x_cvxpy <= -3])
prob.solve()
x_cvxpy.value

array([ 3.00000000e+00, -5.09761577e-17])

In [151]:
m_QP = gp.Model()
x_gurobi = m_QP.addMVar(shape = 2, name = 'x_gurobi')
m_QP.addConstr((- x_gurobi[0] + x_gurobi[1] <= -3 ))
m_QP.addConstr((- x_gurobi[0] - x_gurobi[1] <= -3 ))

m_QP.setObjective(x_gurobi[0], GRB.MINIMIZE)
m_QP.update()
m_QP.optimize()
print()
print(x_gurobi.x)


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x6534bd8a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+00]
Presolve removed 2 rows and 2 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.000000000e+00

[3. 0.]


## QP, Comparison between CVXPY and Gurobi:

In [39]:
# Generate a random non-trivial quadratic program.
m = 15
n = 10
p = 5
np.random.seed(1)
P_gen = np.random.randn(n, n)
P = P_gen.T @ P_gen + np.eye(n)
q = np.random.randn(n)
G = np.random.randn(m, n)
h = G @ np.random.randn(n)
A = np.random.randn(p, n)
b = np.random.randn(p)

In [40]:
# P = np.array([[2, 1], [1, 2]])
# q = np.array([3, 4])
# n = 2

In [43]:
# Via CVXPY:

# Define and solve the CVXPY problem.
x_cvxpy = cp.Variable(n)
prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x_cvxpy, P) + q.T @ x_cvxpy),
                 [G @ x_cvxpy <= h,
                  A @ x_cvxpy == b])
# prob = cp.Problem(cp.Minimize((1/2)*cp.quad_form(x_cvxpy, P) + q.T @ x_cvxpy))
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("A solution x is")
print(x_cvxpy.value)
# print("A dual solution corresponding to the inequality constraints is")
# print(prob.constraints[0].dual_value)


The optimal value is 110.89146278608585
A solution x is
[-1.75612738  0.09726873 -1.57082808 -2.10310111  1.27660026  0.60664563
 -3.22096183  3.30465178  0.55753984 -1.92167322]


In [49]:
# Via Gurobi:

# Initialize model
m_QP = gp.Model()
# Add variables to model
x_gurobi = m_QP.addMVar((n), name = "x", lb = -1000.0, ub = 1000.0)
m_QP.addConstr((G @ x_gurobi <= h))
m_QP.addConstr((A @ x_gurobi == b))

m_QP.setObjective((0.5* x_gurobi.T @ P @ x_gurobi) + (q.T @ x_gurobi), gp.GRB.MINIMIZE)

# m_QP.update()
m_QP.optimize()


Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 23.2.0 23C71)

CPU model: Intel(R) Core(TM) i7-8750H CPU @ 2.20GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 20 rows, 10 columns and 200 nonzeros
Model fingerprint: 0xa58073db
Model has 55 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-03, 3e+00]
  Objective range  [3e-02, 1e+00]
  QObjective range [7e-02, 1e+01]
  Bounds range     [1e+03, 1e+03]
  RHS range        [1e-01, 7e+00]
Presolve time: 0.01s
Presolved: 20 rows, 10 columns, 200 nonzeros
Presolved model has 55 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 9
 AA' NZ     : 4.060e+02
 Factor NZ  : 4.350e+02
 Factor Ops : 8.555e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.65444228e+05 -8.49750212e+06  

In [50]:
np.array([m_QP.getVars()[i].X for i in range(n)])

array([-1.75612735,  0.09726874, -1.57082804, -2.10310108,  1.27660024,
        0.60664563, -3.22096178,  3.30465175,  0.55753985, -1.92167321])

In [51]:
x_cvxpy.value

array([-1.75612738,  0.09726873, -1.57082808, -2.10310111,  1.27660026,
        0.60664563, -3.22096183,  3.30465178,  0.55753984, -1.92167322])

In [48]:
# np.linalg.solve(P, -q)

# Scratch Work:

In [ ]:
# 20*np.linspace(0, 1, 21)[0:]
# np.linspace(toll_arr[0], toll_arr[-3]*T,19)